### Operationalizing ML Models
John Hoff  
Machine Learning Architect  
jhoff@productiveedge.com
# Step 3: Deploy the Model
![Step 1: Prepare](https://drive.google.com/uc?export=view&id=1bgNWs2_1L5QM6UHNYTW94Hwa6rjcspPX)

This step will create a docker image for the trained model and then deploy the image to ACI or AKS.

_**Please Note: This notebook is currently deprecated and does not fully function under the most recent
version of Azure Machine Learning Services.  Native Spark MLlib machine learning models are not being
properly deployed to Docker images to run on ACI and AKS services.  This notebook will be updated as
the issues are sorted out by Databricks and Microsoft.**_

_Please Note: Do not "Run All". There are two different options for model image deployment._

_This notebook will not automatically clean up any services that it creates.  Those steps must be taken manually in the Azure Machine Learning Services UI._

In [2]:
import azureml
import mlflow.azureml

from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import AciWebservice, AksWebservice, Webservice

/databricks/python/lib/python3.5/site-packages/azureml/data/dataset_type_definitions.py:69: DeprecationWarning: FileType Enum is Deprecated in > 1.0.39. Use strings instead.
 category=DeprecationWarning)

In [3]:
#
# The following variables must be customized in order for you to run the notebook.
#
workspace_name = 'ai-saturday'
workspace_location = 'Central US'
resource_group = 'databricks-ai-saturday'
subscription_id = 'f11ca4c4-08ba-41d9-8102-89797ec8f810'

#
# Be sure to update the MLflow Run ID based on the results of Step 2.
#
mlflow_run_id = 'eecf34580dba4ae988c03e220854a971'

#
# The following variables should remain constant
#
aci_webservice_name = 'acisparkmodel'
aks_cluster_name = 'akssparkcluster'
aks_webservice_name = 'akssparkmodel'

## Part 1: Connect to the Azure Machine Learning Service Workspace

In [5]:
workspace = Workspace.create(name = workspace_name,
                             location = workspace_location,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ELLSCR5AU to authenticate.
Interactive authentication successfully completed.

## Part 2: Build the Docker Image for the Model

In [7]:
model_image, azure_model = mlflow.azureml.build_image(model_uri='runs:/' + mlflow_run_id + '/spark_model', 
                                                      workspace=workspace,
                                                      model_name="spark-model",
                                                      image_name="spark-model",
                                                      description="AI Saturday Spark Model",
                                                      synchronous=False)

Registering model spark-model
Creating image

In [8]:
model_image.wait_for_creation(show_output=True)

Running............................................................................................
Succeeded
Image creation operation finished for image spark-model:9, operation "Succeeded"

## Part 3, Option 1: Deploy the Docker Image to ACI

In [10]:
# The default configuration cannot be used, which were designed for simple Python scoring files.
# The Spark models are much more resource intensive.
aci_webservice_deployment_config = AciWebservice.deploy_configuration(cpu_cores=2,
                                                                      memory_gb=1)

aci_webservice = Webservice.deploy_from_image(name=aci_webservice_name,
                                              image=model_image,
                                              deployment_config=aci_webservice_deployment_config,
                                              workspace=workspace)

In [11]:
aci_webservice.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"

In [12]:
# If issues are encountered, the logs can be quickly viewed
# print(aci_webservice.get_logs())

## Part 3, Option 2: Deploy the Docker Image to AKS

In [14]:
aks_config = AksCompute.provisioning_configuration()

aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = aks_config)

In [15]:
aks_target.wait_for_completion(show_output = True)

In [16]:
# The default configuration cannot be used, which were designed for simple Python scoring files.
# The Spark models are much more resource intensive.
aks_webservice_deployment_config = AksWebservice.deploy_configuration(cpu_cores=2,
                                                                      memory_gb=1)

aks_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = aks_webservice_name,
                                               image = model_image,
                                               deployment_config = aks_webservice_deployment_config,
                                               deployment_target = aks_target)

In [17]:
aks_webservice.wait_for_deployment()

In [18]:
# If issues are encountered, the logs can be quickly viewed
# print(aks_webservice.get_logs())